# Activity 4: Mentor Matching & Intervention Recommendations

## Step 1: Load Student & Mentor Data

In [ ]:

import pandas as pd
import numpy as np

# Load student dataset (update path if needed)
students = pd.read_csv('students_scored.csv')

# Load mentor dataset
mentors = pd.read_csv('mentors.csv')

students.head()


## Step 2: Map Cluster to Mentor Type & Intervention

In [ ]:

def intervention_mapping(cluster_label):
    if cluster_label == "At-Risk":
        return ("Wellness", "Wellness + Academic Support")
    elif cluster_label == "High Performer":
        return ("Academic", "Advanced Academic Mentorship")
    else:
        return ("Career", "Career Guidance & Skill Mapping")

students[['Recommended_Mentor_Type', 'Suggested_Intervention']] = students['Cluster_Label'].apply(lambda x: pd.Series(intervention_mapping(x)))

students.head()


## Step 3: Assign Mentors Based on Availability

In [ ]:

def assign_mentor(row):
    eligible = mentors[
        (mentors['Expertise'] == row['Recommended_Mentor_Type']) &
        (mentors['Current_Load'] < mentors['Max_Students'])
    ]
    
    if not eligible.empty:
        selected = eligible.sort_values(by='Current_Load').iloc[0]
        mentors.loc[mentors['Mentor_ID'] == selected['Mentor_ID'], 'Current_Load'] += 1
        return selected['Name']
    else:
        return "No Available Mentor"

students['Assigned_Mentor'] = students.apply(assign_mentor, axis=1)

students.head()


## Step 4: Generate Alerts for High-Risk Students

In [ ]:

def generate_alert(row):
    if row['Cluster_Label'] == "At-Risk" and row['SRI'] >= 7:
        return "Immediate Intervention Required"
    else:
        return "Normal Monitoring"

students['Alert_Status'] = students.apply(generate_alert, axis=1)

students.head()


## Step 5: Final Recommendation Table

In [ ]:

final_table = students[[
    'Student_ID',
    'Cluster_Label',
    'Recommended_Mentor_Type',
    'Assigned_Mentor',
    'Suggested_Intervention',
    'Alert_Status'
]]

final_table.head()

# Save output
final_table.to_csv("Final_Mentor_Assignment.csv", index=False)
